## 3: Subset selection using forward stepwise

In [45]:
#Forward Stepwise #Select best R2

#Create empty dict  
result = {}  
#Create empty list
base_feature_list = []

for i in range(1,len(features1)):
    # Set r2 score = 0 everytimes move to more features
    r2_score_base = 0 
    # select features that not in base list
    for feature in [f for f in features1 if f not in base_feature_list]: 
        #create feature_list
        feature_list = [l for l in base_feature_list] 
        feature_list.append(feature)
        #Start training
        X = train[feature_list]
        y = train['saleprice']
        ss = StandardScaler()
        ss.fit(X)
        X_scale = ss.transform(X)
        lr = LinearRegression()
        lr.fit(X_scale,y)
        r2_score = lr.score(X_scale,y)
        if r2_score > r2_score_base: #if adding features show higher r2 score, store that in dict
            r2_score_base = r2_score
            result_dict = {'feature':feature_list,'r2_score':r2_score}
            result[i] = result_dict
    base_feature_list = result_dict['feature']

result

KeyboardInterrupt: 

In [ ]:
#create dataframe from dict
result_df = pd.DataFrame(result).T

In [ ]:
#calculate cv_score for each row/set of features
cv_score_list = []
for f in result_df['feature']:
    X = train[f]
    y = train['saleprice']
    ss = StandardScaler()
    ss.fit(X)
    X_scale = ss.transform(X)
    lr = LinearRegression()
    lr.fit(X_scale,y)
    cv_r2 = cross_val_score(lr,X_scale,y).mean()
    cv_score_list.append(cv_r2)

result_df['cv_score'] = cv_score_list

In [ ]:
result_df.sort_values('cv_score',ascending=False).head()

In [ ]:
features3 = result[result_df.sort_values('cv_score',ascending=False).head().index[0]]['feature']

In [ ]:
all_model_fit_report(features3)

In [ ]:
# Create X,y
X = train[features3]
y = train['saleprice']
# Standardize
ss = StandardScaler()
ss.fit(X)
X_scale = ss.transform(X)

# LinearRegression
lr = LinearRegression()
lr.fit(X_scale,y)
lr_cv_score = cross_val_score(lr,X_scale,y).mean()
lr_cv_rmse = np.abs(cross_val_score(lr,X_scale,y,scoring='neg_root_mean_squared_error')).mean()

In [ ]:
lr_cv_rmse

In [ ]:
#select subset of test data
X_test = test[features3]
#standardize
X_test_scale = ss.transform(X_test)
#predict and assign to saleprice column
submit['SalePrice'] = lr.predict(X_test_scale)

In [ ]:
submit.head()

In [ ]:
#Print csv file
submit.to_csv('../Model/3_forward_stepwise_lr.csv',index=False)

### 3.1 Forward stepwise + train-test split

In [147]:
#Forward Stepwise #Select best R2
result = {}  #Create empty dict  
base_feature_list = [] #Create empty list
for i in range(1,len(features1)):
    r2_score_base = 0 # Set r2 score = 0 everytimes move to more features
    for feature in [f for f in features1 if f not in base_feature_list]: # select features that not in base list
        #create feature list
        feature_list = [l for l in base_feature_list] #create feature_list
        feature_list.append(feature) #append 1 features
        #create X,y
        X = train[feature_list]
        y = train['saleprice']
        #do train-test split
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)
        #standardize
        ss = StandardScaler()
        ss.fit(X_train)
        X_train_scale = ss.transform(X_train)
        X_test_scale = ss.transform(X_test)
        #Do linear regression
        lr = LinearRegression()
        lr.fit(X_train_scale,y_train)
        #calculate r2 test
        r2_test = lr.score(X_test_scale,y_test)
        if r2_test > r2_score_base: #if adding features show higher r2 score of testing set, store that in dict
            r2_score_base = r2_test
            result_dict = {'feature':feature_list,'r2_test':r2_test}
            result[i] = result_dict
    base_feature_list = result_dict['feature']

result

{1: {'feature': ['overall_qual gr_liv_area'], 'r2_test': 0.7733968055593904},
 2: {'feature': ['overall_qual gr_liv_area', 'total_bsmt_sf garage_area'],
  'r2_test': 0.85406508324468},
 3: {'feature': ['overall_qual gr_liv_area',
   'total_bsmt_sf garage_area',
   'ms_subclass_180'],
  'r2_test': 0.8815388671360405},
 4: {'feature': ['overall_qual gr_liv_area',
   'total_bsmt_sf garage_area',
   'ms_subclass_180',
   'ms_subclass_50'],
  'r2_test': 0.8900197289269925},
 5: {'feature': ['overall_qual gr_liv_area',
   'total_bsmt_sf garage_area',
   'ms_subclass_180',
   'ms_subclass_50',
   'year_built gr_liv_area'],
  'r2_test': 0.8843786438023644},
 6: {'feature': ['overall_qual gr_liv_area',
   'total_bsmt_sf garage_area',
   'ms_subclass_180',
   'ms_subclass_50',
   'year_built gr_liv_area',
   'bsmt_unf_sf'],
  'r2_test': 0.8934446130940912},
 7: {'feature': ['overall_qual gr_liv_area',
   'total_bsmt_sf garage_area',
   'ms_subclass_180',
   'ms_subclass_50',
   'year_built gr_li

In [148]:
#create dataframe from dict
result_df = pd.DataFrame(result).T
result_df.sort_values('r2_test',ascending=False)

,feature,r2_test
217,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9485
252,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9480
240,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9478
249,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9475
204,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9474
...,...,...
4,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.8900
5,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.8844
3,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.8815
2,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.8541


In [149]:
#calculate cv_score for each row/set of features
cv_score_list = []
for f in result_df['feature']:
    X = train[f]
    y = train['saleprice']
    ss = StandardScaler()
    ss.fit(X)
    X_scale = ss.transform(X)
    lr = LinearRegression()
    lr.fit(X_scale,y)
    cv_r2 = cross_val_score(lr,X_scale,y).mean()
    cv_score_list.append(cv_r2)

result_df['cv_score'] = cv_score_list

In [150]:
result_df.sort_values('cv_score',ascending=False)

,feature,r2_test,cv_score
141,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9449,0.9171
33,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9261,0.9023
32,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9255,0.9018
31,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9253,0.9010
25,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9201,0.8893
...,...,...,...
91,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9244,-15328506768125533922263040.0000
99,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9293,-16428504894381293278593024.0000
351,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9347,-21006067227582125507084288.0000
88,"[overall_qual gr_liv_area, total_bsmt_sf garag...",0.9359,-37175966171331231496536064.0000


In [151]:
features3_1 = result[result_df.sort_values('cv_score',ascending=False).head().index[0]]['feature']

In [152]:
len(features3_1)

141

In [153]:
all_model_fit_report(features3_1)

     
         LinearRegression
         R2 Score on training data: 0.937
         RMSE on training data :19902.2179
         Estimated R2 Score on unseen data: 0.9171
         Estimated RMSE on unseen data: 22730.9416

         Ridge
         R2 Score on training data: 0.9359
         RMSE on training data :20075.2053
         Estimated R2 Score on unseen data: 0.9164
         Estimated RMSE on unseen data: 22833.5949

         Lasso
         R2 Score on training data: 0.9267
         RMSE on training data :21455.8457
         Estimated R2 Score on unseen data: 0.9157
         Estimated RMSE on unseen data: 22932.401   

         ElasticNet
         Opimal l1 ration: 1.0
         R2 Score on training data: 0.9267
         RMSE on test training :21460.7463
         Estimated R2 Score on unseen data: 0.9157
         Estimated RMSE on unseen data: 22932.1665         
        


'LinearRegression'

In [154]:
# Create X,y
X = train[features3_1]
y = train['saleprice']
# Standardize
ss = StandardScaler()
ss.fit(X)
X_scale = ss.transform(X)

# LinearRegression
lr = LinearRegression()
lr.fit(X_scale,y)
lr_cv_score = cross_val_score(lr,X_scale,y).mean()
lr_cv_rmse = np.abs(cross_val_score(lr,X_scale,y,scoring='neg_root_mean_squared_error')).mean()

In [155]:
lr_cv_rmse

22730.94160806355

In [156]:
#select subset of test data
X_test = test[features3_1]
#standardize
X_test_scale = ss.transform(X_test)
#predict and assign to saleprice column
submit['SalePrice'] = lr.predict(X_test_scale)

In [157]:
submit.head()

,Id,SalePrice
0,2658,166305.9246
1,2718,151882.0906
2,2414,204683.3417
3,1989,131280.4973
4,625,168611.8878


In [158]:
#Print csv file
submit.to_csv('../Model/3_1_forward_stepwise_traintest_lr.csv',index=False)

In [159]:
#LassoCV
lassocv = LassoCV(n_alphas=200)
lassocv.fit(X_scale,y)
#Lasso
lasso = Lasso(alpha=lassocv.alpha_)
lasso.fit(X_scale,y)
lasso_cv_score = cross_val_score(lasso,X_scale,y).mean()
lasso_cv_rmse = np.abs(cross_val_score(lasso,X_scale,y,scoring='neg_root_mean_squared_error')).mean()

In [160]:
lasso_cv_rmse

22932.400955390964

In [161]:
#select subset of test data
X_test = test[features3_1]
#standardize
X_test_scale = ss.transform(X_test)
#predict and assign to saleprice column
submit['SalePrice'] = lasso.predict(X_test_scale)

In [162]:
submit.head()

,Id,SalePrice
0,2658,158095.2245
1,2718,154077.1827
2,2414,203758.3501
3,1989,122340.2753
4,625,171012.8410


In [163]:
#Print csv file
submit.to_csv('../Model/3_1_forward_stepwise_traintest_lasso.csv',index=False)

In [ ]:
## 3: Subset selection using forward stepwise

#Forward Stepwise #Select best R2

#Create empty dict  
result = {}  
#Create empty list
base_feature_list = []

for i in range(1,len(features1)):
    # Set r2 score = 0 everytimes move to more features
    r2_score_base = 0 
    # select features that not in base list
    for feature in [f for f in features1 if f not in base_feature_list]: 
        #create feature_list
        feature_list = [l for l in base_feature_list] 
        feature_list.append(feature)
        #Start training
        X = train[feature_list]
        y = train['saleprice']
        ss = StandardScaler()
        ss.fit(X)
        X_scale = ss.transform(X)
        lr = LinearRegression()
        lr.fit(X_scale,y)
        r2_score = lr.score(X_scale,y)
        if r2_score > r2_score_base: #if adding features show higher r2 score, store that in dict
            r2_score_base = r2_score
            result_dict = {'feature':feature_list,'r2_score':r2_score}
            result[i] = result_dict
    base_feature_list = result_dict['feature']

result

#create dataframe from dict
result_df = pd.DataFrame(result).T

#calculate cv_score for each row/set of features
cv_score_list = []
for f in result_df['feature']:
    X = train[f]
    y = train['saleprice']
    ss = StandardScaler()
    ss.fit(X)
    X_scale = ss.transform(X)
    lr = LinearRegression()
    lr.fit(X_scale,y)
    cv_r2 = cross_val_score(lr,X_scale,y).mean()
    cv_score_list.append(cv_r2)

result_df['cv_score'] = cv_score_list

result_df.sort_values('cv_score',ascending=False).head()

features3 = result[result_df.sort_values('cv_score',ascending=False).head().index[0]]['feature']

all_model_fit_report(features3)

# Create X,y
X = train[features3]
y = train['saleprice']
# Standardize
ss = StandardScaler()
ss.fit(X)
X_scale = ss.transform(X)

# LinearRegression
lr = LinearRegression()
lr.fit(X_scale,y)
lr_cv_score = cross_val_score(lr,X_scale,y).mean()
lr_cv_rmse = np.abs(cross_val_score(lr,X_scale,y,scoring='neg_root_mean_squared_error')).mean()

lr_cv_rmse

#select subset of test data
X_test = test[features3]
#standardize
X_test_scale = ss.transform(X_test)
#predict and assign to saleprice column
submit['SalePrice'] = lr.predict(X_test_scale)

submit.head()

#Print csv file
submit.to_csv('../Model/3_forward_stepwise_lr.csv',index=False)

### 3.1 Forward stepwise + train-test split

#Forward Stepwise #Select best R2
result = {}  #Create empty dict  
base_feature_list = [] #Create empty list
for i in range(1,len(features1)):
    r2_score_base = 0 # Set r2 score = 0 everytimes move to more features
    for feature in [f for f in features1 if f not in base_feature_list]: # select features that not in base list
        #create feature list
        feature_list = [l for l in base_feature_list] #create feature_list
        feature_list.append(feature) #append 1 features
        #create X,y
        X = train[feature_list]
        y = train['saleprice']
        #do train-test split
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)
        #standardize
        ss = StandardScaler()
        ss.fit(X_train)
        X_train_scale = ss.transform(X_train)
        X_test_scale = ss.transform(X_test)
        #Do linear regression
        lr = LinearRegression()
        lr.fit(X_train_scale,y_train)
        #calculate r2 test
        r2_test = lr.score(X_test_scale,y_test)
        if r2_test > r2_score_base: #if adding features show higher r2 score of testing set, store that in dict
            r2_score_base = r2_test
            result_dict = {'feature':feature_list,'r2_test':r2_test}
            result[i] = result_dict
    base_feature_list = result_dict['feature']

result

#create dataframe from dict
result_df = pd.DataFrame(result).T
result_df.sort_values('r2_test',ascending=False)

#calculate cv_score for each row/set of features
cv_score_list = []
for f in result_df['feature']:
    X = train[f]
    y = train['saleprice']
    ss = StandardScaler()
    ss.fit(X)
    X_scale = ss.transform(X)
    lr = LinearRegression()
    lr.fit(X_scale,y)
    cv_r2 = cross_val_score(lr,X_scale,y).mean()
    cv_score_list.append(cv_r2)

result_df['cv_score'] = cv_score_list

result_df.sort_values('cv_score',ascending=False)

features3_1 = result[result_df.sort_values('cv_score',ascending=False).head().index[0]]['feature']

len(features3_1)

all_model_fit_report(features3_1)

# Create X,y
X = train[features3_1]
y = train['saleprice']
# Standardize
ss = StandardScaler()
ss.fit(X)
X_scale = ss.transform(X)

# LinearRegression
lr = LinearRegression()
lr.fit(X_scale,y)
lr_cv_score = cross_val_score(lr,X_scale,y).mean()
lr_cv_rmse = np.abs(cross_val_score(lr,X_scale,y,scoring='neg_root_mean_squared_error')).mean()

lr_cv_rmse

#select subset of test data
X_test = test[features3_1]
#standardize
X_test_scale = ss.transform(X_test)
#predict and assign to saleprice column
submit['SalePrice'] = lr.predict(X_test_scale)

submit.head()

#Print csv file
submit.to_csv('../Model/3_1_forward_stepwise_traintest_lr.csv',index=False)

#LassoCV
lassocv = LassoCV(n_alphas=200)
lassocv.fit(X_scale,y)
#Lasso
lasso = Lasso(alpha=lassocv.alpha_)
lasso.fit(X_scale,y)
lasso_cv_score = cross_val_score(lasso,X_scale,y).mean()
lasso_cv_rmse = np.abs(cross_val_score(lasso,X_scale,y,scoring='neg_root_mean_squared_error')).mean()

lasso_cv_rmse

#select subset of test data
X_test = test[features3_1]
#standardize
X_test_scale = ss.transform(X_test)
#predict and assign to saleprice column
submit['SalePrice'] = lasso.predict(X_test_scale)

submit.head()

#Print csv file
submit.to_csv('../Model/3_1_forward_stepwise_traintest_lasso.csv',index=False)